In [1]:
# Importacion de librerias
# Tratamiento de datos
# ==========================================
import pandas as pd
# Graficas
# ==========================================
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import requests

import warnings
warnings.filterwarnings("ignore")

### Importamos Data - SECOP I ADICIONES

In [2]:
# Define the API endpoint URL
url = "https://www.datos.gov.co/resource/7fix-nd37.json?$limit=1500000"

# Send an HTTP GET request to the API endpoint
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    print("Data received from API:")
else:
    print(f"Request failed with status code {response.status_code}")

Data received from API:


In [3]:
# Set the display option to show all columns
pd.set_option('display.max_columns', None)
secop_adiciones = pd.DataFrame(data=data)
secop_adiciones.head(3)

,id_adjudicacion,adicion_en_valor,adicion_en_dias,adicion_en_meses,fecha_firma
0,8709817,1725000,45,0,2019-10-31T00:00:00.000
1,9392317,0,0,6,2020-06-17T00:00:00.000
2,5958205,7920000,30,0,2017-06-30T00:00:00.000


## Importar tabla secop I desde BigQuery

In [4]:
from utils.bq_conn import bq_conn_file

bq_client = bq_conn_file(project_id="dsa-miad-365616")

In [5]:
query = '''
SELECT
    *
FROM `Proyecto.DatosSecopI`
ORDER BY anno_cargue_secop DESC
'''
secop_I = bq_client.execute_query(query=query)

In [6]:
secop_I.head(3)

,uid,anno_cargue_secop,id_modalidad,modalidad_de_contratacion,estado_del_proceso,id_objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipios_ejecucion_,departamento_ejecucion,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,tipo_doc_representante_legal,identific_representante_legal,plazo_de_ejec_del_contrato_,rango_de_ejec_del_contrato,prorrogado,tiempo_adiciones_en_dias,cuantia_contrato,con_adicion_pptal,valor_total_de_adiciones,valor_contrato_con_adiciones,moneda,objeto_del_contrato_a_la,ruta_proceso_en_secop_i
0,191389136418143424,2019,13,contratacion minima cuantia,celebrado,50000000,contratar servicio suministro elementos materi...,suministro,facatativa,facatativa,cundinamarca,CPMMSFFAMCNo01DE2019,50000000,50000000,E,productos de uso final,5022,productos de cereales legumbres,502213,harina productos de molinos,8143424,nit de persona juridica,900791326,inversiones ocazo,cedula de ciudadania,1022363447,330,D,0,0,50000000,0,0,50000000,peso colombiano,contratar compra elementos materia prima para ...,{'url': 'https://www.contratos.gov.co/consulta...
1,1912101041779226352,2019,12,contratacion directa ley de,celebrado,53000000,adquisicion uniformes para dotar personal cuer...,prestacion de servicios,bogota,bogota,bogota,DIRGENERALCONTRATACIONDIRECTA184DE2019,22980096902,22980096902,E,productos de uso final,5310,ropa,531027,uniformes,9226352,nit de persona juridica,860020227,fondo rotatorio de la policia forpo,nit de persona juridica,860020227,660,D,1,30,22980096902,0,0,22980096902,peso colombiano,adquisicion uniformes para dotar personal cuer...,{'url': 'https://www.contratos.gov.co/consulta...
2,1912101443259265538,2019,12,contratacion directa ley de,celebrado,55000000,contratar diseno produccion tarjetas presentac...,prestacion de servicios,bogota,bogota,bogota,DIRGENERALCONTRATACIONDIRECTA188DE2019,35998452,35998452,E,productos de uso final,5510,medios impresos,551015,publicaciones impresas,9265538,nit de persona juridica,830001113,imprenta nacional de colombia,nit de persona juridica,830001113,31,D,1,40,35998452,0,0,35998452,peso colombiano,contratar diseno produccion tarjetas presentac...,{'url': 'https://www.contratos.gov.co/consulta...


#### Comparar las adiciones del SECOP I con los datos extraidos en SECOP I

In [10]:
id_adj_secopI = secop_I[['id_adjudicacion']]
secop_adiciones_inpec = pd.merge(secop_adiciones,id_adj_secopI, on='id_adjudicacion')
secop_adiciones_inpec

,id_adjudicacion,adicion_en_valor,adicion_en_dias,adicion_en_meses,fecha_firma
0,6039384,250000,0,0,2017-10-24T00:00:00.000
1,6649432,336000,0,0,2017-11-28T00:00:00.000
2,5373642,514300,0,0,2016-12-13T00:00:00.000
3,5729629,9864335,0,0,2017-10-13T00:00:00.000
4,5706720,10808640,0,0,2017-04-24T00:00:00.000
...,...,...,...,...,...
3155,5950913,859835.12,0,0,2017-05-24T00:00:00.000
3156,3429905,1262858,0,0,2014-07-28T00:00:00.000
3157,5518186,275800,0,3,2017-01-24T00:00:00.000
3158,7054541,6666666.67,79,0,2018-09-13T00:00:00.000


In [11]:
secop_I.loc[secop_I.id_adjudicacion == "6039384"]

,uid,anno_cargue_secop,id_modalidad,modalidad_de_contratacion,estado_del_proceso,id_objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipios_ejecucion_,departamento_ejecucion,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,tipo_doc_representante_legal,identific_representante_legal,plazo_de_ejec_del_contrato_,rango_de_ejec_del_contrato,prorrogado,tiempo_adiciones_en_dias,cuantia_contrato,con_adicion_pptal,valor_total_de_adiciones,valor_contrato_con_adiciones,moneda,objeto_del_contrato_a_la,ruta_proceso_en_secop_i
7497,171366222056039384,2017,13,contratacion minima cuantia,liquidado,72000000,realizar control manejo vectores artropodos ro...,prestacion de servicios,duitama,duitama,boyaca,105MC0242017,2310000,2310000,F,servicios,7210,servicios de mantenimiento reparaciones de con...,721021,control de plagas,6039384,nit de persona juridica,900436116,tcv fumigaciones ltda,cedula de ciudadania,24079567,180,D,0,0,1980000,1,250000,2230000,peso colombiano,realizar control manejo vectores artropodos ro...,{'url': 'https://www.contratos.gov.co/consulta...
